# Bibliotecas

In [23]:
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

# Importação

In [24]:
# Dados obtidos do site da B3
# https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/
df_cota_ago = pd.read_csv('COTAHIST_M082022.csv', sep=';')
df_cota_set = pd.read_csv('COTAHIST_M092022.csv', sep=';')
df_cota_out = pd.read_csv('COTAHIST_M102022.csv', sep=';')
# Gera um DataFrame com os últimos 3 meses
df_cota = pd.concat([df_cota_ago, df_cota_set, df_cota_out])
print(df_cota)

        DATAPREGAO   CODNEG  PREULT  TOTNEG
0         20220801   A1BB34    3888       1
1         20220801   A1CR34    6734     113
2         20220801   A1DM34   43645       4
3         20220801   A1EG34    2259       5
4         20220801   A1EN34   31424      60
...            ...      ...     ...     ...
181801    20221018  ITSAI99     237       1
181802    20221019  ITSAI99     242       3
181803    20221021  ITSAI99     268       1
181804    20221024  ITSAI99     259       1
181805    20221025  ITSAI99     253       4

[564011 rows x 4 columns]


# Tratamentos dos dados

In [25]:
df_cota.dtypes
df_cota['PREULT'] = df_cota['PREULT']/100 
df_cota.head(10)

,DATAPREGAO,CODNEG,PREULT,TOTNEG
0,20220801,A1BB34,38.88,1
1,20220801,A1CR34,67.34,113
2,20220801,A1DM34,436.45,4
3,20220801,A1EG34,22.59,5
4,20220801,A1EN34,314.24,60
5,20220801,A1IV34,42.24,4
6,20220801,A1KA34,40.92,1
7,20220801,A1LB34,1247.17,3
8,20220801,A1LL34,51.00,2
9,20220801,A1MB34,376.80,1


## Seleciona o ativo aleatório

In [26]:
# Seleciona os 100 ativos mais negociados nos últimos 3 meses
mais_negociados = """SELECT a.CODNEG,
                            SUM(a.TOTNEG) SOMATOTNEG
                     FROM df_cota a
                     GROUP BY a.CODNEG
                     ORDER BY SUM(a.TOTNEG) DESC
                     LIMIT 100
         ;"""

df_mais_neg = pysqldf(mais_negociados)

#  Exibe o resultado
print(df_mais_neg)

    CODNEG  SOMATOTNEG
0    VALE3     4062773
1   BOVA11     3870049
2    PETR4     3793421
3    MGLU3     3353350
4    ITUB4     3321356
..     ...         ...
95  VGIA11      697771
96   CBAV3      691429
97   BRAP4      680851
98  BBAS3F      680191
99  ENGI11      668636

[100 rows x 2 columns]


In [30]:
# Filtra ativo aleatorio
df_ativo_aleatorio = df_mais_neg['CODNEG']
df_ativo_aleatorio = df_ativo_aleatorio.sample(n=1, replace=False)
print(df_ativo_aleatorio)

68    INBR32
Name: CODNEG, dtype: object


In [34]:
# Filtra os dados da B3 para pegar somente o ativo aleatório selecionado
df_tendencia = df_cota.merge(df_ativo_aleatorio, how='inner', on='CODNEG')
print(df_tendencia)

    DATAPREGAO  CODNEG  PREULT  TOTNEG
0     20220829  INBR32   21.50   12550
1     20220830  INBR32   20.80   16714
2     20220831  INBR32   20.80   22991
3     20220916  INBR32   20.89   23312
4     20220901  INBR32   22.33   14734
5     20220909  INBR32   21.30   37176
6     20220919  INBR32   20.89   12891
7     20220902  INBR32   21.00   13545
8     20220923  INBR32   20.92   30034
9     20220912  INBR32   21.86   24548
10    20220905  INBR32   21.00    6807
11    20220920  INBR32   21.63   13158
12    20220913  INBR32   21.57   10687
13    20220906  INBR32   19.75   30433
14    20220921  INBR32   21.66   10112
15    20220926  INBR32   20.00   17357
16    20220914  INBR32   22.40   15451
17    20220908  INBR32   20.56   19675
18    20220922  INBR32   22.34   11560
19    20220915  INBR32   21.48   13498
20    20220930  INBR32   17.40   41954
21    20220927  INBR32   18.99   19017
22    20220928  INBR32   18.02   72957
23    20220929  INBR32   17.84   86671
24    20221014  INBR32   

## Análise da tendência

In [32]:
# Verifica a variação do preço de fechamento
consulta = """SELECT d.TIPO,
                     d.TENDENCIA,
                     d.TENDENCIA_CONFIRMADA,
                     count(*) QUANTIDADE
              FROM ( 
                        SELECT c.CODNEG,
                        c.DATAPREGAO,
                        c.PREULT,
                        c.TIPO,
                        -- DESCONSIDERA SE A LINHA ANTERIOR FOR DA MESMA TENDENCIA PRA NAO CONTABILIZAR DOBRADO
                        case when LAG(TENDENCIA,1,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TENDENCIA
                                        then 'NAO' else  TENDENCIA END AS TENDENCIA,
                        -- DESCONSIDERA SE A LINHA ANTERIOR TAMBEM FOR DE TENDENCIA CONFIRMADA PRA NAO CONTABILIZAR DOBRADO
                        case when LAG(TENDENCIA_CONFIRMADA,1,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TENDENCIA_CONFIRMADA
                                        then 'NAO' else  TENDENCIA_CONFIRMADA END AS TENDENCIA_CONFIRMADA
                        FROM (
                                SELECT  b.*,
                                        -- VERIFICA SE EXISTE A TENDENCIA NOS PROXIMOS 4 DIAS
                                        CASE WHEN LEAD(TIPO,1,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TIPO
                                                AND LEAD(TIPO,2,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TIPO
                                                        AND LEAD(TIPO,3,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TIPO
                                                                AND LEAD(TIPO,4,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TIPO
                                                                        THEN 'SIM' 
                                        ELSE 'NAO' 
                                        END AS TENDENCIA,
                                        -- VERIFICA SE A TENDENCIA SE CONFIRMOU NOS PROXIMOS 5 DIAS
                                        CASE WHEN LEAD(TIPO,1,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TIPO
                                                AND LEAD(TIPO,2,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TIPO
                                                        AND LEAD(TIPO,3,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TIPO
                                                                AND LEAD(TIPO,4,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TIPO
                                                                        AND LEAD(TIPO,5,0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO) = TIPO
                                                                                THEN 'SIM' 
                                        ELSE 'NAO' 
                                        END AS TENDENCIA_CONFIRMADA
                                FROM   (
                                        SELECT
                                                A.CODNEG,
                                                A.DATAPREGAO,
                                                A.PREULT,
                                                -- VERIFICA O TIPO DA TENDENCIA, COMPARANDO COM A LINHA ANTERIOR
                                                CASE WHEN a.PREULT > (LAG(PREULT, 1, 0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO)) THEN 'ALTA' 
                                                WHEN a.PREULT < (LAG(PREULT, 1, 0) OVER (PARTITION BY CODNEG ORDER BY DATAPREGAO)) THEN 'BAIXA' 
                                                else 'NEUTRA'
                                                END AS TIPO        
                                        FROM df_tendencia a
                                        ORDER BY a.CODNEG, a.DATAPREGAO
                                        ) b
                                ) c 
                        ) d
                GROUP BY d.TIPO, d.TENDENCIA, d.TENDENCIA_CONFIRMADA
         ;"""

df_tendencia = pysqldf(consulta)

#  Exibe o resultado
print(df_tendencia)

     TIPO TENDENCIA TENDENCIA_CONFIRMADA  QUANTIDADE
0    ALTA       NAO                  NAO          18
1   BAIXA       NAO                  NAO          22
2   BAIXA       SIM                  SIM           1
3  NEUTRA       NAO                  NAO           3
